In [2]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import random

import joblib
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import plotly.graph_objects as go


In [4]:

# === Settings
output_dir = r"output"
os.makedirs(output_dir, exist_ok=True)

# === Reproducibility
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

set_seed(42)

# === Load Data
df = pd.read_csv(r"resources/data.csv")
X = df.drop(columns=["FoS", "SeismicFoS"]).values
y = df["FoS"].values  # Change to df["SeismicFoS"].values for seismic model

# === Preprocessing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


scaler_path = os.path.join(output_dir, "ABC_ANN_FoS_Scaler.pkl")
joblib.dump(scaler, scaler_path)
print(f"✅ Scaler saved to: {scaler_path}")


X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)


✅ Scaler saved to: output/ABC_ANN_FoS_Scaler.pkl


In [3]:

# === ANN Model
class ANN(nn.Module):
    def __init__(self, input_dim, h1, h2, h3):
        super(ANN, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, h1), nn.ReLU(),
            nn.Linear(h1, h2), nn.ReLU(),
            nn.Linear(h2, h3), nn.ReLU(),
            nn.Linear(h3, 1)
        )
    def forward(self, x):
        return self.model(x)

# === Evaluation Function
# === Evaluation Function (with model return)
def evaluate_model(h1, h2, h3, lr):
    model = ANN(X_train.shape[1], h1, h2, h3)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    for epoch in range(100):
        model.train()
        for xb, yb in train_loader:
            optimizer.zero_grad()
            pred = model(xb)
            loss = criterion(pred, yb)
            loss.backward()
            optimizer.step()
    model.eval()
    with torch.no_grad():
        y_train_pred = model(X_train_tensor).numpy()
        y_test_pred = model(X_test_tensor).numpy()

    metrics = {
        "r2_train": r2_score(y_train, y_train_pred),
        "r2_test": r2_score(y_test, y_test_pred),
        "rmse": np.sqrt(mean_squared_error(y_test, y_test_pred)),
        "mae": mean_absolute_error(y_test, y_test_pred),
        "mse": mean_squared_error(y_test, y_test_pred),
        "rmse_train": np.sqrt(mean_squared_error(y_train, y_train_pred)),
        "mae_train": mean_absolute_error(y_train, y_train_pred),
        "mse_train": mean_squared_error(y_train, y_train_pred)
    }
    return model, metrics


In [5]:

# === ABC Parameters
num_food_sources = 10
limit = 5
cycles = 10

# === Initialize Food Sources
def init_food_sources():
    return [{
        "h1": random.randint(16, 128),
        "h2": random.randint(16, 128),
        "h3": random.randint(16, 128),
        "lr": round(random.uniform(0.0001, 0.01), 5),
        "trial": 0
    } for _ in range(num_food_sources)]

food_sources = init_food_sources()
best_solution = None
best_score = -np.inf
history = []



In [6]:
best_solution = None
best_score = -np.inf
best_model_state = None  # To store the best model weights
history = []

for cycle in range(cycles):
    print(f"\n🔄 Cycle {cycle+1}")
    for fs in food_sources:
        model, metrics = evaluate_model(fs["h1"], fs["h2"], fs["h3"], fs["lr"])
        fs.update(metrics)

        # Save the best model found so far
        if metrics["r2_test"] > best_score:
            best_score = metrics["r2_test"]
            best_solution = fs.copy()
            best_model_state = model.state_dict()  # Save model weights

    # Onlooker Bees Phase
    fitnesses = [fs["r2_test"] for fs in food_sources]
    total_fitness = sum(fitnesses)
    probs = [f / total_fitness for f in fitnesses]
    new_sources = []
    for _ in range(num_food_sources):
        selected = np.random.choice(food_sources, p=probs)
        candidate = selected.copy()
        idx = random.choice(["h1", "h2", "h3", "lr"])
        if idx != "lr":
            candidate[idx] = random.randint(16, 128)
        else:
            candidate[idx] = round(random.uniform(0.0001, 0.01), 5)
        new_sources.append(candidate)
    food_sources = new_sources

    # Scout Bees Phase
    for fs in food_sources:
        fs["trial"] += 1
        if fs["trial"] > limit:
            fs.update({
                "h1": random.randint(16, 128),
                "h2": random.randint(16, 128),
                "h3": random.randint(16, 128),
                "lr": round(random.uniform(0.0001, 0.01), 5),
                "trial": 0
            })

    print(f"🐝 Best so far: ({best_solution['h1']}, {best_solution['h2']}, {best_solution['h3']}) | "
          f"lr={best_solution['lr']} | R² Test = {best_solution['r2_test']:.6f}")
    history.append((cycle + 1, best_solution["r2_test"]))
# === Save Best Model
model_save_path = os.path.join(output_dir, "ABC_ANN_FoS_BestModel.pth")
torch.save({
    'model_state_dict': best_model_state,
    'input_dim': X_train.shape[1],
    'h1': best_solution['h1'],
    'h2': best_solution['h2'],
    'h3': best_solution['h3'],
}, model_save_path)
print(f"\n💾 Best model saved to: {model_save_path}")



🔄 Cycle 1
🐝 Best so far: (102, 110, 85) | lr=0.00096 | R² Test = 0.915230

🔄 Cycle 2
🐝 Best so far: (64, 20, 19) | lr=0.00103 | R² Test = 0.923560

🔄 Cycle 3
🐝 Best so far: (69, 44, 26) | lr=0.00593 | R² Test = 0.927838

🔄 Cycle 4
🐝 Best so far: (69, 44, 26) | lr=0.00593 | R² Test = 0.927838

🔄 Cycle 5
🐝 Best so far: (69, 44, 26) | lr=0.00593 | R² Test = 0.927838

🔄 Cycle 6
🐝 Best so far: (119, 84, 30) | lr=0.00761 | R² Test = 0.928619

🔄 Cycle 7
🐝 Best so far: (119, 84, 30) | lr=0.00761 | R² Test = 0.928619

🔄 Cycle 8
🐝 Best so far: (119, 84, 30) | lr=0.00761 | R² Test = 0.928619

🔄 Cycle 9
🐝 Best so far: (119, 84, 30) | lr=0.00761 | R² Test = 0.928619

🔄 Cycle 10
🐝 Best so far: (119, 84, 30) | lr=0.00761 | R² Test = 0.928619

💾 Best model saved to: output/ABC_ANN_FoS_BestModel.pth


In [7]:
# === Load Saved Model Later
checkpoint = torch.load(model_save_path)
loaded_model = ANN(checkpoint['input_dim'], checkpoint['h1'], checkpoint['h2'], checkpoint['h3'])
loaded_model.load_state_dict(checkpoint['model_state_dict'])
loaded_model.eval()


ANN(
  (model): Sequential(
    (0): Linear(in_features=9, out_features=119, bias=True)
    (1): ReLU()
    (2): Linear(in_features=119, out_features=84, bias=True)
    (3): ReLU()
    (4): Linear(in_features=84, out_features=30, bias=True)
    (5): ReLU()
    (6): Linear(in_features=30, out_features=1, bias=True)
  )
)